In [ ]:
# import libraries and datasets

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# load the dataset

# TODO import correct test set
behavior = pd.read_csv("data/MINDsmall_dev/behaviors.tsv", sep="\t", header=None, names=["Impression ID", "User ID", "Time", "History", "Impressions"])
news = pd.read_csv("data/MINDsmall_dev/news.tsv", sep="\t", header=None, names=["News ID", "Category", "Subcategory", "Title", "Abstract", "URL", "Title Entities", "Abstract Entities", "Title Topics", "Abstract Topics"])

In [ ]:
# Constants
theta = 0.5

In [ ]:
# Takes in dataframe with articles and a score, returns dataframe with articles and [1/0], 1 if score above threshold, 0 otherwise
def binarize_score(scored_data):
    binarized_data = []
    for index, row in scored_data.iterrows():
        if row["Score"] > theta:
            binarized_data.append((row["News ID"], 1))
        else:
            binarized_data.append((row["News ID"], 0))
    return pd.DataFrame(binarized_data)


# TODO: Compare binary with actual clicks. Eval function
def DCGi():
    
    pass